In [1]:
from processing import convert_to_mne
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

matplotlib.use("Qt5Agg")

path = '../runs/marco/test_0/pretraining.csv'

rescale = 1e6
fs = 250
chs = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8"]
columns = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "AccX", "AccY", "AccZ", "GyroX", "GyroY", "GyroZ",
            "CNT", "BAT", "VALID", "DeltaTime", "Trigger"]

data = pd.read_csv(filepath_or_buffer=path, names=columns, delimiter='\t')

print(data.head())

minutes = len(data)/(60*fs)
seconds = (len(data)/fs) % 60 
print(f'Recording duration: {int(minutes)} minutes and {int(seconds)} seconds')

trigger = data.iloc[1:, -1].to_numpy(dtype=np.float64)
eeg = data.iloc[1:, 0:8].to_numpy(dtype=np.float64)
raw_data = convert_to_mne(eeg, trigger, rescale=rescale, fs=fs, chs=chs, recompute=False)

              Fz           C3             Cz             C4             Pz  \
0  656472.562500  750000.0625  548680.437500  338597.468750  543826.625000   
1  245311.218750  721747.8750   -5735.726074 -576059.125000  750000.062500   
2  125166.023438  649578.0000  -18569.740234 -599866.562500  750000.062500   
3  276962.812500  658194.6250  187449.156250  -97709.226562  303487.812500   
4  325688.968750  712552.5625  235800.359375 -163764.171875    3638.238281   

             PO7           Oz            PO8      AccX      AccY      AccZ  \
0  626551.437500  750000.0625  750000.062500 -0.071777  0.967041  0.353027   
1  308482.875000  671162.6875  750000.062500 -0.065430  0.979980  0.350586   
2  247410.484375  556451.4375  513618.343750 -0.063477  0.980713  0.345215   
3  294142.187500  535008.6875  279445.531250 -0.060791  0.980225  0.344727   
4  402658.812500  513673.0625 -235630.046875 -0.061768  0.981689  0.347412   

   GyroX  GyroY  GyroZ        CNT  BAT  VALID     DeltaTime  T

In [2]:
# Compute PSD
Pxx = raw_data.compute_psd(fmin=0, fmax=fs/2)
Pxx.plot()
plt.show()

Effective window size : 8.192 (s)


In [3]:
filtered = raw_data.copy() # the method filters the signal in-place, so this time I
                      # want to preserve the original signal and filter just a
                      # temporary copy of it

# remove power line noise
filtered.notch_filter(50) 
filtered.notch_filter(60) 
# Apply band-pass filtering
filtered.filter(1,30) 

pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
pxx_filt.plot()
plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

In [4]:
filtered.plot()
plt.show()

Using matplotlib as 2D backend.
Channels marked as bad:
none


### Channels Interpolation
Previously marked bad channels can be interpolated

In [7]:
## Interpolate bad channels
interp = filtered.copy()
interp.interpolate_bads()
interp.plot()
plt.show()

Setting channel interpolation method to {'eeg': 'spline'}.


C:\Users\Gianni\AppData\Local\Temp\ipykernel_41288\3975301400.py:3: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  interp.interpolate_bads()


Channels marked as bad:
none


### Cleaning data using ASR
Full example available: https://github.com/DiGyt/asrpy/blob/main/example.ipynb

In [6]:
## take the first 30 seconds of the data
training_data = interp.copy()
training_data.crop(tmin=0, tmax=20)
training_data.plot()
plt.show()

Channels marked as bad:
none


In [8]:
import asrpy

asr = asrpy.ASR(sfreq=fs, cutoff=15)
asr.fit(training_data)
cleaned = asr.transform(interp)
cleaned.plot()
plt.show()

Channels marked as bad:
none
